In [1]:
#Criando um dataframe de exemplo
df = sc.parallelize([['00000000000', 'Amanda', 22, True],['11111111111', 'Amanda', 555, True],
                     ['22222222222', 'Marcos', 45, False],['33333333333', 'Fernando', 18, False]]).toDF(("DocumentRegistration", "Name", "Age", "isEnabled"))

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1647453599178_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------+---+---------+
|DocumentRegistration|    Name|Age|isEnabled|
+--------------------+--------+---+---------+
|         00000000000|  Amanda| 22|     true|
|         11111111111|  Amanda|555|     true|
|         22222222222|  Marcos| 45|    false|
|         33333333333|Fernando| 18|    false|
+--------------------+--------+---+---------+

In [3]:
from great_expectations.data_context import BaseDataContext
from great_expectations.data_context.types.base import DataContextConfig

bucket = "your-bucket"
prefix_expectations = "ExpectationsStore/"
prefix_validations = "ValidationsStore/"
prefix_data_docs = "DataDocs/"

#Configurando o Data Context
project_config = DataContextConfig(
    config_version=2,
    plugins_directory=None,
    config_variables_file_path=None,
    datasources={
        "my_spark_datasource": {
            "data_asset_type": {
                "class_name": "SparkDFDataset",
                "module_name": "great_expectations.dataset",
            },
            "class_name": "SparkDFDatasource",
            "module_name": "great_expectations.datasource",
            "batch_kwargs_generators": {},
        }
    },
    stores={
        "expectations_S3_store": {
            "class_name": "ExpectationsStore",
            "store_backend": {
                "class_name": "TupleS3StoreBackend",
                "bucket": bucket,
                "prefix": prefix_expectations,
            },
        },
        "validations_S3_store": {
            "class_name": "ValidationsStore",
            "store_backend": {
                "class_name": "TupleS3StoreBackend",
                "bucket": bucket,
                "prefix": prefix_validations,
            },
        },
        "evaluation_parameter_store": {"class_name": "EvaluationParameterStore"},
    },
    expectations_store_name="expectations_S3_store",
    validations_store_name="validations_S3_store",
    evaluation_parameter_store_name="evaluation_parameter_store",
    data_docs_sites={
        "s3_site": {
            "class_name": "SiteBuilder",
            "store_backend": {
                "class_name": "TupleS3StoreBackend",
                "bucket": bucket,
                "prefix": prefix_data_docs,
            },
            "site_index_builder": {
                "class_name": "DefaultSiteIndexBuilder",
                "show_cta_footer": True,
                "validation_results_limit": 50,
            },
        }
    },
    validation_operators={
        "action_list_operator": {
            "class_name": "ActionListValidationOperator",
            "action_list": [
                {
                    "name": "store_validation_result",
                    "action": {"class_name": "StoreValidationResultAction"},
                },
                {
                    "name": "store_evaluation_params",
                    "action": {"class_name": "StoreEvaluationParametersAction"},
                },
                {
                    "name": "update_data_docs",
                    "action": {"class_name": "UpdateDataDocsAction"},
                },
            ],
        }
    },
    anonymous_usage_statistics={"enabled": False},
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
context = BaseDataContext(project_config=project_config)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
#Associa o contexto criado no passo anterior ao arquivo "tangerines_example_expectation_suite.json". Este arquivo deve ser inserido no path de Expectations definido no DataContext

expectation_suite_name = 'tangerines_example_expectation_suite'
expectation_suite = context.get_expectation_suite(expectation_suite_name)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
batch_kwargs = {
    "datasource": "my_spark_datasource",
    "dataset": df
}

#A função get batch vai associar a expectation com o dataset a ser validado
batch = context.get_batch(
    batch_kwargs=batch_kwargs,
    expectation_suite_name=expectation_suite
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
#Ao executar essa linha, tanto o json quanto o data docs relacionados a validação do dataframe serão criados
results = context.run_validation_operator("action_list_operator", assets_to_validate=[batch])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…